In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import pandas as pd
import duck_db_helper
import duckdb

We do simple feature generation to get the odds for the players team and the odds agaings the players team by looking at if the player was at home or not.

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
conn = duckdb.connect('../data/analytical_sandboxes/analytical_sandbox_zone.db')
tables = duck_db_helper.get_tables(conn)
df_list = []
df_list_names = []
for table_name in tables: 
    df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
    df_list.append(df)
    df_list_names.append(table_name)
conn.close()
df = df_list[0]

TODO if this is the only feature generation we are going to do, i suggest we remove all the game from the analytical sandbox. They just clutter up the tables and we never use them.

In [4]:
home_odds_col = 'AvgH'
draw_odds_col = 'AvgD'
away_odds_col = 'AvgA'

df['odds_against'] = 0.0
df['odds_for'] = 0.0

# Fill in odds columns based on whether the player was home or not
df.loc[df['was_home'] == 1, 'odds_for'] = df.loc[df['was_home'] == 1, home_odds_col]
df.loc[df['was_home'] == 0, 'odds_for'] = df.loc[df['was_home'] == 0, away_odds_col]
df.loc[df['was_home'] == 1, 'odds_against'] = df.loc[df['was_home'] == 1, away_odds_col]
df.loc[df['was_home'] == 0, 'odds_against'] = df.loc[df['was_home'] == 0, home_odds_col]

df.drop(columns=[home_odds_col, away_odds_col], inplace=True)
df.rename(columns={'AvgD': 'odds_draw'}, inplace=True)

In [5]:
df.loc[df.was_home==0].head(5)

,team_x,opp_team_name,was_home,name,position,assists,bonus,bps,clean_sheets,creativity,element,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,yellow_cards,red_cards,round,saves,selected,threat,total_points,value,FTHG,FTAG,FTR,odds_draw,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW,odds_against,odds_for
3658,Bournemouth,Man City,0,Lewis Cook,MID,0,0,7,0,0.6,67,3,0,0.4,3.0,63,0,0,0,0,0,2,0,5294,0,2,50,4,0,H,12.72,1017.8,E,5.0,0.0,20.8,11.9,1.07,37.57
3659,Bournemouth,Man City,0,Jordan Zemura,DEF,0,0,0,0,0.0,76,0,0,0.0,0.0,0,0,0,0,0,0,2,0,17896,0,0,45,4,0,H,12.72,1017.8,E,5.0,0.0,20.8,11.9,1.07,37.57
3660,Bournemouth,Man City,0,Ryan Christie,MID,0,0,11,0,1.3,69,3,0,1.2,8.6,62,0,0,0,0,0,2,0,6375,2,2,55,4,0,H,12.72,1017.8,E,5.0,0.0,20.8,11.9,1.07,37.57
3661,Bournemouth,Man City,0,Dominic Solanke,FWD,0,0,0,0,0.0,66,0,0,0.0,0.0,0,0,0,0,0,0,2,0,98173,0,0,60,4,0,H,12.72,1017.8,E,5.0,0.0,20.8,11.9,1.07,37.57
3662,Bournemouth,Man City,0,Lloyd Kelly,DEF,0,0,12,0,0.2,73,4,0,1.7,16.6,90,0,0,0,0,0,2,0,148416,0,0,45,4,0,H,12.72,1017.8,E,5.0,0.0,20.8,11.9,1.07,37.57


In [6]:
conn = duckdb.connect('../data/feature_generation/feature_generation.db')
duck_db_helper.create_table('feature_1', df, conn)
conn.close()